In [1]:
import os
import time
import re
import json
import pandas as pd
from tqdm import tqdm
from itertools import cycle
import google.generativeai as genai
from dotenv import load_dotenv

In [2]:
# ===============================
# 📂 2. LOAD DATA 
# ===============================

file_path = "C:/Users/mloai/Sentimental_Analysis_App/Notebooks/cleaned_full_dataset.csv"

# Load the data
df = pd.read_csv(file_path, encoding='utf-8', dtype=str)

text_column = "CLEAN_ANSWER"


In [3]:
# === Load API Keys from .env ===
load_dotenv()
API_KEYS = [os.getenv(f"GEMINI_KEY_{i}") for i in range(1, 10)]
API_KEYS = [key for key in API_KEYS if key]
if not API_KEYS:
    raise ValueError("❌ No Gemini API keys found. Please define GEMINI_KEY_1 to GEMINI_KEY_9 in your .env file.")

key_cycle = cycle(API_KEYS)
usage_counter = {key: {'success': 0, 'fail': 0} for key in API_KEYS}

In [4]:
# === Prompt Template ===
PROMPT_TEMPLATE = {
    "analysis_instructions": {
        "required_fields": {
            "final_sentiment": ["Positive", "Neutral", "Negative"],
            "sarcasm": ["Yes", "No"],
            "suggestion": ["Yes", "No"],
            "suggestion_text": ["Brief summary of the user's suggestion (if any)", "None"],
            "suggestion_type": ["Complaint", "Feature Request", "Operational Improvement", "Other", "None"],
            "emotion": [
                "joy", "gratitude", "satisfaction", "confusion", "frustration",
                "disappointment", "worry", "anger", "resentment"
            ],
            "tone": [
                "enthusiastic", "grateful", "polite", "professional",
                "informal", "urgent", "frustrated", "disappointed", "angry", "sarcastic"
            ],
            "urgency_flag": ["Yes", "No"],
            "impact_level": ["Low", "Medium", "High"],
            "responsible_party": [
                "delivery team", "customer service", "billing",
                "technical support", "management", "sales", "None"
            ],
            "action_flag": ["Yes", "No"]
        },
        "guidelines": {
            "1": "Focus only on the customer's feedback and implied meaning.",
            "2": "Ignore greetings, emojis, or irrelevant text.",
            "3": "Identify emotional and tonal cues from wording, even if subtle.",
            "4": "Label 'suggestion' as Yes only if the user is proposing an improvement or asking for something.",
            "5": "Use 'suggestion_type' to clarify the nature of the suggestion.",
            "6": "Only assign 'sarcasm' if there are clear indicators like irony, mocking, or conflicting tone and sentiment.",
            "7": "Set 'urgency_flag' to 'Yes' if the customer expresses urgency using time-sensitive language in English or Arabic (e.g., 'immediately', 'as soon as possible', 'now', 'urgent', 'delay', 'still not resolved', OR Arabic phrases like 'حالاً', 'فوراً', 'بسرعة', 'أنتظر', 'لا يزال لم يتم', 'تأخير'). Also set to 'Yes' if the tone is angry, frustrated, or disappointed AND the impact level is 'High'. Otherwise, set to 'No'."
,
            "8": "Set 'impact_level' to High if the issue seriously affects the user's experience or outcome.",
            "9": "Mark 'action_flag' as Yes if the feedback requires a response, escalation, or operational fix.",
            "10": "If no party is clearly responsible, set 'responsible_party' to 'None'.",
            "11": "Ensure 'suggestion_text' is concise and not a full copy of the original input."
        }
    },
    "task": "Based on the customer review below, extract all required fields according to the instructions above.",
    "customer_review": ""
}


In [5]:
def call_gemini_api(api_key, customer_text):
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel("gemini-2.5-flash")

    prompt = PROMPT_TEMPLATE.copy()
    prompt["customer_review"] = customer_text

    prompt_text = (
        f"{json.dumps(prompt['analysis_instructions'], indent=2, ensure_ascii=False)}\n\n"
        f"Customer Review: {customer_text}"
    )

    try:
        response = model.generate_content(prompt_text)
        raw_output = response.text.strip()

        # ✅ Remove markdown backticks (```json ... ```)
        if raw_output.startswith("```"):
            raw_output = re.sub(r"^```json|```$", "", raw_output.strip(), flags=re.IGNORECASE).strip()

        parsed = json.loads(raw_output)
        print(f"✅ Success with key {api_key[:5]}: {parsed}")
        return parsed

    except Exception as e:
        print(f"⚠️ API Error with key {api_key[:5]}...: {e}")
        return "API_ERROR"


In [6]:
def rotate_gemini_call(row, text_column="CLEAN_ANSWER"):
    customer_text = row.get(text_column, "")
    if not customer_text:
        return {}

    for _ in range(len(API_KEYS)):
        api_key = next(key_cycle)
        result = call_gemini_api(api_key, customer_text)
        if result != "API_ERROR":
            usage_counter[api_key]['success'] += 1
            time.sleep(6.5)  # breathing time between successful calls
            return result
        usage_counter[api_key]['fail'] += 1
        time.sleep(1.5)
    return {}

In [7]:
# === Run Pipeline ===
def run_gemini_pipeline():

    df = pd.read_csv(file_path, encoding="utf-8", dtype=str)

    # Optional row limiting for testing
    row_limit = input("🔢 How many rows do you want to process (e.g. 100)? Leave blank for all: ").strip()
    if row_limit.isdigit():
        df = df.head(int(row_limit))

    results = []
    print(f"\n🔍 Starting Gemini Analysis on {len(df)} rows...\n")
    with tqdm(total=len(df), desc="🔍 Gemini Analysis", unit="row") as pbar:
        for _, row in df.iterrows():
            result = rotate_gemini_call(row, text_column)
            results.append(result)
            pbar.update(1)

    results_df = pd.json_normalize(results)
    final_df = pd.concat([df.reset_index(drop=True), results_df], axis=1)
    final_df.to_csv("gemini_sentiment_output.csv", index=False, encoding="utf-8-sig")
    print("✅ Saved as gemini_sentiment_output.csv")

In [9]:
 run_gemini_pipeline()

🔢 How many rows do you want to process (e.g. 100)? Leave blank for all:  500



🔍 Starting Gemini Analysis on 500 rows...



🔍 Gemini Analysis:   0%|          | 0/500 [00:00<?, ?row/s]

✅ Success with key AIzaS: {'final_sentiment': 'Negative', 'sarcasm': 'No', 'suggestion': 'Yes', 'suggestion_text': 'Suggests contracting with a high-quality and reputable company to ensure customer and time respect.', 'suggestion_type': 'Operational Improvement', 'emotion': 'disappointment', 'tone': 'polite', 'urgency_flag': 'Yes', 'impact_level': 'High', 'responsible_party': 'management', 'action_flag': 'Yes'}


🔍 Gemini Analysis:   0%|          | 1/500 [00:22<3:03:04, 22.01s/row]

...

✅ Success with key AIzaS: {'final_sentiment': 'Negative', 'sarcasm': 'No', 'suggestion': 'Yes', 'suggestion_text': 'Call center should operate 24 hours', 'suggestion_type': 'Operational Improvement', 'emotion': 'frustration', 'tone': 'frustrated', 'urgency_flag': 'No', 'impact_level': 'High', 'responsible_party': 'customer service', 'action_flag': 'Yes'}


🔍 Gemini Analysis: 100%|█████████▉| 499/500 [2:30:54<00:19, 19.25s/row]

✅ Success with key AIzaS: {'final_sentiment': 'Positive', 'sarcasm': 'No', 'suggestion': 'No', 'suggestion_text': 'None', 'suggestion_type': 'None', 'emotion': 'satisfaction', 'tone': 'grateful', 'urgency_flag': 'No', 'impact_level': 'Low', 'responsible_party': 'None', 'action_flag': 'No'}


🔍 Gemini Analysis: 100%|██████████| 500/500 [2:31:04<00:00, 18.13s/row]

✅ Saved as gemini_sentiment_output.csv
